# DSCI 100 Project Proposal: Pulsar Stars

## Introduction

## Preliminary Exploratory Data Analysis

In [11]:
# Import packages
library(tidyverse)

In [14]:
# Read csv file
htru_data <- read_csv('data/HTRU_2.csv', col_names = FALSE)

# Adding column names: 
# 1. Mean of the integrated profile
# 2. Standard deviation of the integrated profile
# 3. Excess kurtosis of the integrated profile 
# 4. Skewness of the integrated profile
# 5. Mean of the DM-SNR curve
# 6. Standard deviation of the DM-SNR curve
# 7. Excess kurtosis of the DM-SNR curve
# 8. Skewness of the DM-SNR curve
# 9. Class
colnames(htru_data) <- c('mean_int_prof', 'sd_int_prof', 'kurtosis_int_prof', 'skew_int_prof', 'mean_DMSNR', 'sd_DMSNR', 'kurtosis_DMSNR', 'skew_DMSNR', 'class')
htru_data <- htru_data %>% 
    mutate(class = as.factor(class))
head(htru_data)

Rows: 17898 Columns: 9

── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (9): X1, X2, X3, X4, X5, X6, X7, X8, X9


ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.



mean_int_prof,sd_int_prof,kurtosis_int_prof,skew_int_prof,mean_DMSNR,sd_DMSNR,kurtosis_DMSNR,skew_DMSNR,class
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
140.56250,55.68378,-0.23457141,-0.6996484,3.199833,19.11043,7.975532,74.24222,0
102.50781,58.88243,0.46531815,-0.5150879,1.677258,14.86015,10.576487,127.39358,0
103.01562,39.34165,0.32332837,1.0511644,3.121237,21.74467,7.735822,63.17191,0
136.75000,57.17845,-0.06841464,-0.6362384,3.642977,20.95928,6.896499,53.59366,0
88.72656,40.67223,0.60086608,1.1234917,1.178930,11.46872,14.269573,252.56731,0
93.57031,46.69811,0.53190485,0.4167211,1.636288,14.54507,10.621748,131.39400,0


## Methods

## Expected Outcomes and Significance